En este script, vamos a escribir la lógica para que Spark reciba y procese el texto de una serier de artículos publicados en 2016. Nuestro objetivo es recibir dichos articulos como DStreams a través del puerto 9009 y comenzar a procesarlos. Vamos separar dichos artículos por palabras y generar un "Bag of Words" con todas ellas. En términos formales, un "Bag Of Words" no es más que un multiconjunto donde cada elemento es una palabra. Es decir, un conjunto de palabras dónde además aparece la multiplicidad de cada una. 

En términos más prácticos, un Bag of Words nos permite llevar un conteo de qué palabras son más o menos frecuentes en los artículos que vayamos leyendo. Una vez Spark haya leído y procesado todos los artículos, habrá generado un extenso Bag of Words que podremos procesar de diferentes maneras. En lo que respecta a este trabajo, únicamente mostraremos las 10 palabras más comunes y las 10 menos comunes. El "Bag of Words" generado se serializará y se almacenará en disco para poder ser utilizado en otras ocasiones.

In [1]:
import pickle

bow = {}
def update_bag_of_words(rdd):
    if rdd.collect() == ['END']:
        # Obtenemos los 10 items más comunes
        aux1 = sorted(bow, key = bow.get, reverse = True)[:10]
        print(f"Palabras más comunes -> {aux1}")
        # Obtenemos los 10 items menos comunes
        aux2 = sorted(bow, key = bow.get)[:10]
        print(f"Palabras menos comnes -> {aux2}")
        # Guardamos el Bag of Words con pickle
        with open('bow.pickle', 'wb') as handle:
            pickle.dump(bow, handle)
    for word in rdd.collect():
        bow[word] = bow.get(word, 0) + 1

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext

# Creamos configuración spark
conf = SparkConf()
conf.setAppName("StreamApp")
# Creamos contexto Spark a partir de la conf
sc = SparkContext(conf = conf)
sc.setLogLevel("WARN")
# Creamos contexto streaming desde el contexto spark
ssc = StreamingContext(sc, 1)
# Establecemos punto de control para permitir la recuperación de RDD
ssc.checkpoint("chekpoint_App")
# Comenzamos a leer datos del puerto 9009
dataStream = ssc.socketTextStream("localhost", 9009)
# Obtenemos las palabras del artículo
words = dataStream.flatMap(lambda line: line.split(" "))
words.pprint()
# Actualizamos el Bag of Words del artículo leído
words.foreachRDD(update_bag_of_words)
# Comenzamos computación del streaming
ssc.start()
# Esperamos a que termine la transmisión
ssc.awaitTermination(10)

-------------------------------------------
Time: 2022-06-20 14:20:48
-------------------------------------------
MADRID.-
Bodybell
ha
concluido
el
Expediente
de
Regulación
de
Empleo
...

-------------------------------------------
Time: 2022-06-20 14:20:49
-------------------------------------------
VienaActualizado:21/03/2016
20:38hLa
OPEP
consideró
hoy
que
la
estrategia
de
congelar
...

-------------------------------------------
Time: 2022-06-20 14:20:50
-------------------------------------------
BARCELONA.
Susan
George,
la
filósofa
y
politóloga
reconocida
por
su
...

-------------------------------------------
Time: 2022-06-20 14:20:51
-------------------------------------------
Las
Palmas
de
Gran
CanariaActualizado:10/01/2016
16:51hUn
veterano
agente
de
la
...

-------------------------------------------
Time: 2022-06-20 14:20:52
-------------------------------------------
En
agosto
publiqué
una
columna
en
The
Guardian
con
el
...

-------------------------------------------
Time

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 38014)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 239, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/